In [ ]:
from transformers import pipeline
pipe = pipeline("text-classification", model = "ProsusAI/finbert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")



In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
sentiment = pd.read_excel("/content/complete_nifty.xlsx")
sentiment
test = sentiment["News Article"]
date = sentiment["Date"]
test.shape

(18122,)

In [ ]:
def finbert(example):
  encoded_text = tokenizer(example, return_tensors= 'pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
      'positive': scores[0].round(3),
      'negative': scores[1].round(3),
      'neutral': scores[2].round(3)
  }
  return(scores_dict)

In [ ]:
finbert_results = []  # Initialize an empty list to store the results

for i in tqdm(range(len(test))):
    result = finbert(test[i])
    finbert_results.append(result)  # Append the result to the list

# Now finbert_results contains the result of finbert(test[i]) for each i



  0%|          | 0/18122 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(finbert_results)
merged = df.join(test, how = "left")
merged

,positive,negative,neutral,News Article
0,0.158,0.011,0.831,Top 10 stocks based on technical parameters wh...
1,0.061,0.014,0.924,"Sell BPCL, SBI; buy NMDC, Hindalco: Ashwani Gu..."
2,0.953,0.022,0.025,"VECV sales up 49.5% at 6,087 units in December"
3,0.030,0.373,0.597,"GST Sentimeter: McDonalds, Lifestyle & HUL's s..."
4,0.859,0.015,0.126,IndusInd Bank raises $500 mn from overseas len...
...,...,...,...,...
18117,0.733,0.019,0.248,TMT rebars demand to grow over next 3-4 months...
18118,0.182,0.058,0.760,"No respite for TCS, Infosys & other IT firms i..."
18119,0.025,0.043,0.932,Darlings of Dalal Street: The stocks most owne...
18120,0.948,0.019,0.033,Larsen & Toubro bags large orders in Middle East


In [ ]:
for index, row in merged.iterrows():
    if row["positive"] > row["neutral"] and row["positive"] > row["negative"]:
        merged.at[index, "sentiment"] = "Positive"
    elif row["negative"] > row["neutral"] and row["negative"] > row["positive"]:
        merged.at[index, "sentiment"] = "Negative"
    elif row["negative"] < row["neutral"] and row["neutral"] > row["positive"]:
        merged.at[index, "sentiment"] = "Neutral"

final = merged.join(date, how = "left")
final

,positive,negative,neutral,News Article,sentiment,Date
0,0.158,0.011,0.831,Top 10 stocks based on technical parameters wh...,Neutral,2018-01-01
1,0.061,0.014,0.924,"Sell BPCL, SBI; buy NMDC, Hindalco: Ashwani Gu...",Neutral,2018-01-01
2,0.953,0.022,0.025,"VECV sales up 49.5% at 6,087 units in December",Positive,2018-01-01
3,0.030,0.373,0.597,"GST Sentimeter: McDonalds, Lifestyle & HUL's s...",Neutral,2018-01-01
4,0.859,0.015,0.126,IndusInd Bank raises $500 mn from overseas len...,Positive,2018-01-01
...,...,...,...,...,...,...
18117,0.733,0.019,0.248,TMT rebars demand to grow over next 3-4 months...,Positive,2023-12-18
18118,0.182,0.058,0.760,"No respite for TCS, Infosys & other IT firms i...",Neutral,2023-12-18
18119,0.025,0.043,0.932,Darlings of Dalal Street: The stocks most owne...,Neutral,2023-12-20
18120,0.948,0.019,0.033,Larsen & Toubro bags large orders in Middle East,Positive,2023-12-28


In [ ]:
final = final[["Date","News Article","positive", "negative", "neutral"]]
final

,Date,News Article,positive,negative,neutral
0,2018-01-01,Top 10 stocks based on technical parameters wh...,0.158,0.011,0.831
1,2018-01-01,"Sell BPCL, SBI; buy NMDC, Hindalco: Ashwani Gu...",0.061,0.014,0.924
2,2018-01-01,"VECV sales up 49.5% at 6,087 units in December",0.953,0.022,0.025
3,2018-01-01,"GST Sentimeter: McDonalds, Lifestyle & HUL's s...",0.030,0.373,0.597
4,2018-01-01,IndusInd Bank raises $500 mn from overseas len...,0.859,0.015,0.126
...,...,...,...,...,...
18117,2023-12-18,TMT rebars demand to grow over next 3-4 months...,0.733,0.019,0.248
18118,2023-12-18,"No respite for TCS, Infosys & other IT firms i...",0.182,0.058,0.760
18119,2023-12-20,Darlings of Dalal Street: The stocks most owne...,0.025,0.043,0.932
18120,2023-12-28,Larsen & Toubro bags large orders in Middle East,0.948,0.019,0.033


In [ ]:
final_mean =final.groupby("Date")[["positive", "negative", "neutral"]].mean().reset_index()
final_mean

,Date,positive,negative,neutral
0,2018-01-01,0.412208,0.078375,0.509333
1,2018-01-02,0.472050,0.067850,0.459800
2,2018-01-03,0.425741,0.047111,0.527111
3,2018-01-04,0.216200,0.106000,0.677850
4,2018-01-05,0.294154,0.188615,0.517462
...,...,...,...,...
1864,2023-12-15,0.067500,0.154500,0.778000
1865,2023-12-18,0.286000,0.037500,0.676500
1866,2023-12-20,0.025000,0.043000,0.932000
1867,2023-12-28,0.948000,0.019000,0.033000


In [ ]:
for index , row in final.iterrows():
  if row["sentiment"] == "Neutral":
    merged.at[index, "Sentiment_value"] = 0
  elif row["sentiment"] == "Positive":
    merged.at[index, "Sentiment_value"] = 1
  elif row["sentiment"] == "Negative":
    merged.at[index, "Sentiment_value"] = -1
final

,positive,negative,neutral,News Article,sentiment,Sentiment_value,Date
0,0.158,0.011,0.831,Top 10 stocks based on technical parameters wh...,Neutral,0.0,2018-01-01
1,0.061,0.014,0.924,"Sell BPCL, SBI; buy NMDC, Hindalco: Ashwani Gu...",Neutral,0.0,2018-01-01
2,0.953,0.022,0.025,"VECV sales up 49.5% at 6,087 units in December",Positive,1.0,2018-01-01
3,0.030,0.373,0.597,"GST Sentimeter: McDonalds, Lifestyle & HUL's s...",Neutral,0.0,2018-01-01
4,0.859,0.015,0.126,IndusInd Bank raises $500 mn from overseas len...,Positive,1.0,2018-01-01
...,...,...,...,...,...,...,...
18117,0.733,0.019,0.248,TMT rebars demand to grow over next 3-4 months...,Positive,1.0,2023-12-18
18118,0.182,0.058,0.760,"No respite for TCS, Infosys & other IT firms i...",Neutral,0.0,2023-12-18
18119,0.025,0.043,0.932,Darlings of Dalal Street: The stocks most owne...,Neutral,0.0,2023-12-20
18120,0.948,0.019,0.033,Larsen & Toubro bags large orders in Middle East,Positive,1.0,2023-12-28


In [ ]:
final = final[["Date","News Article", "sentiment","Sentiment_value"]]
final

,Date,News Article,sentiment,Sentiment_value
0,2018-01-01,Top 10 stocks based on technical parameters wh...,Neutral,0.0
1,2018-01-01,"Sell BPCL, SBI; buy NMDC, Hindalco: Ashwani Gu...",Neutral,0.0
2,2018-01-01,"VECV sales up 49.5% at 6,087 units in December",Positive,1.0
3,2018-01-01,"GST Sentimeter: McDonalds, Lifestyle & HUL's s...",Neutral,0.0
4,2018-01-01,IndusInd Bank raises $500 mn from overseas len...,Positive,1.0
...,...,...,...,...
18117,2023-12-18,TMT rebars demand to grow over next 3-4 months...,Positive,1.0
18118,2023-12-18,"No respite for TCS, Infosys & other IT firms i...",Neutral,0.0
18119,2023-12-20,Darlings of Dalal Street: The stocks most owne...,Neutral,0.0
18120,2023-12-28,Larsen & Toubro bags large orders in Middle East,Positive,1.0


In [ ]:
sent_mean = final.groupby("Date")["Sentiment_value"].sum().reset_index()
sent_mean


,Date,Sentiment_value
0,2018-01-01,9.0
1,2018-01-02,9.0
2,2018-01-03,11.0
3,2018-01-04,2.0
4,2018-01-05,2.0
...,...,...
1864,2023-12-15,0.0
1865,2023-12-18,1.0
1866,2023-12-20,0.0
1867,2023-12-28,1.0


In [ ]:
negative_sentiment = []
for index, row in sent_mean.iterrows():
    if row["Sentiment_value"] < 0:
        negative_sentiment.append(row["Date"])
len(negative_sentiment)

296

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Export the DataFrame to an Excel file
final_mean.to_excel('/content/drive/My Drive/sentiment_grouped_mean.xlsx', index=False)

Mounted at /content/drive
